In [8]:
import h5py

with h5py.File("data/sift/sift-128-euclidean.hdf5", "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    distances = list(f.keys())[0]
    neighbors = list(f.keys())[1]
    test = list(f.keys())[2]
    train = list(f.keys())[3]

    # Get the data
    train = list(f[train])
    distances = list(f[distances])
    neighbors = list(f[neighbors])
    test = list(f[test])

.hdf5
Keys: <KeysViewHDF5 ['distances', 'neighbors', 'test', 'train']>


In [12]:
# we have 10000 test vectors
# for each of those vectors we have their 100 closest neighbors in order from closest to farthest
# we also have their distances away from each of these 100 neighbors
len(test[0])

128

In [1]:
# read in all the quantized vectors and codebooks
# create the approximate vectors in place (retaining indices)
# take that X and do np.argmin(np.linalg.norm(X - queries))
from collections import defaultdict

def readQuantAndCodes(rootDirectory, M):
    quantFile = rootDirectory + "quantized.txt"
    codesFile = rootDirectory + "codebooks.txt"
    quantizedVectors = []
    codebooks = defaultdict(list)
    with open(quantFile, "r") as f:
        lines = f.readlines()
        for line in lines:
            quantizedVectors.append([int(x) for x in line.split(" ")])

    with open(codesFile, "r") as f:
        lines = f.readlines()
        codebook_index = 0
        for line in lines:
            centroids_in_column = []
            whole_line_as_ints = [float(x) for x in line.split(" ")]
            index = 0
            for i in range(256):
                centroid = []
                for j in range(M):
                    centroid.append(whole_line_as_ints[index])
                    index += 1
                codebooks[codebook_index].append(centroid)
            codebook_index += 1

    return quantizedVectors, codebooks


for M in [50]:
    quantizedVectors, codebooks = readQuantAndCodes(f"data/word2vec/testing/testing_M{M}/", M)
    full_quantized = []
    for vector in quantizedVectors:
        actual_vector = []
        for i in range(len(vector)):
            for code in codebooks[i][vector[i]]:
                actual_vector.append(code)
        full_quantized.append(actual_vector)


In [6]:
import numpy as np

np.array(full_quantized)

[0.751, 0.174, 0.219, 1.721, -0.937, -1.687, 0.905, 2.756, -1.246, -0.936, 1.139, 0.694, -0.446, -0.469, 0.176, 1.884, 3.43, -0.211, 1.094, 0.182, 0.831, 0.247, 2.08, -0.513, -0.321, 0.711, -1.464, -0.552, -1.052, -1.873, -0.448, 1.896, 0.85, 0.129, 1.833, -1.435, -1.563, 0.187, -0.466, 0.511, -0.274, 0.538, -2.065, 1.83, -0.777, -0.52, -3.648, 1.071, 1.042, -1.531]
